Models
And now - this colab unveils the heart (or the brains?) of the transformers library - the models:

This should run nicely on a low-cost or free T4 box

In [ ]:
%pip install requests torch bitsandbytes transformers sentencepiece accelerate
#%pip install -U bitsandbytes

In [7]:
#from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

Inicia Sesión en Hugging Face.

In [8]:
# Opción 1: Google Colab (comentado para uso local)
# from google.colab import userdata
# hf_token = userdata.get('HUGGINGFACE_API_KEY')

# Opción 2: Desde archivo .env (para uso local/Docker)
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='/workspace/.env', override=True)
hf_token = os.getenv('HUGGINGFACE_API_KEY')

# Remove 'Bearer ' prefix if present, as login expects the raw token
if hf_token and hf_token.startswith('Bearer '):
    hf_token = hf_token.replace('Bearer ', '')
# Strip any leading/trailing whitespace that might have been accidentally included
if hf_token:
    hf_token = hf_token.strip()

print(f"Token cargado: {'Connected' if hf_token else 'Error'}")
#login(hf_token, add_to_git_credential_=True)

Token cargado: Connected


In [9]:
login(hf_token)

In [10]:
# Modelos a utilizar.

LLAMA = "meta-llama/Llama-3.1-8B-Instruct"
PHI4 = "microsoft/Phi-3-mini-4k-instruct"
GEMMA3 = "google/gemma-3-4b-it"
QWEN = "Qwen/Qwen3-4B-Instruct-2507"
MIXTRAL = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [11]:
content_system = "Eres un asistente util"
content_user = "Cuentame un chiste divertido para una sala llena de cientificos de datos"

messages = [
    {"role": "system", "content": content_system},
    {"role": "user", "content": content_user}
]

print(messages)

[{'role': 'system', 'content': 'Eres un asistente util'}, {'role': 'user', 'content': 'Cuentame un chiste divertido para una sala llena de cientificos de datos'}]


**Acceder a Llama desde Meta.**

Para poder usar Llama, meta requiere que se firmen los terminos de servicio.

[Repositorio Llama](https://huggingface.co/collections/meta-llama/metas-llama-31-models-and-evals)


In [12]:
# Quantization config - Esto permite cargar el modelo en la memoria y utilizar menos memoria
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,    # Se cargaran en 4 bits
    bnb_4bit_use_double_quant=True,   # Cuantifica los pesos 2 veces.
    bnb_4bit_compute_dtype=torch.bfloat16, # Los productos y operaciones tienen 16 bits
    bnb_4bit_quant_type="nf4"
)

### ¿Qué hace la Cuantización?

**Cuantización** es el proceso de reducir la precisión numérica de los pesos del modelo:

- **Modelo Original (FP32/FP16)**: Cada peso usa 32 o 16 bits → ~16GB para Llama-3.1-8B
- **Modelo Cuantizado (4-bit)**: Cada peso usa solo 4 bits → ~4-5GB para el mismo modelo

**¿Qué se descarga y qué se cuantiza?**

1. **Descarga inicial**: Se descargan los pesos originales (FP16/FP32) desde HuggingFace → `~/.cache/huggingface/hub/`
2. **Cuantización en memoria**: BitsAndBytes cuantiza los pesos **durante la carga** en RAM/GPU
3. **Resultado**: El modelo cuantizado existe solo en memoria durante la ejecución

**NO** se guarda el modelo cuantizado en disco automáticamente. Cada vez que ejecutas el código:
- Se leen los pesos originales del caché
- Se cuantizan nuevamente en memoria
- Se liberan al ejecutar `del model`

**Configuración de Cuantización**:
- `load_in_4bit=True`: Reduce de 16/32 bits → 4 bits (75% menos memoria)
- `bnb_4bit_use_double_quant=True`: Cuantiza también los parámetros de cuantización (ahorro adicional)
- `bnb_4bit_compute_dtype=torch.bfloat16`: Las operaciones matemáticas usan 16 bits (balance precisión/velocidad)
- `bnb_4bit_quant_type="nf4"`: Tipo de cuantización optimizado para redes neuronales

**Trade-offs**:
- ✅ Ventaja: 75% menos memoria GPU/RAM
- ✅ Ventaja: Permite ejecutar modelos grandes en hardware limitado
- ⚠️ Desventaja: ~2-5% pérdida de precisión en respuestas
- ⚠️ Desventaja: Re-cuantización en cada carga (~30-60 segundos)

In [ ]:
# Tokenizer
# Usar variable de entorno HF_HOME en lugar de ruta hardcodeada
import os
cache_dir = os.getenv('HF_HOME', '/root/.cache/huggingface')

tokenizer = AutoTokenizer.from_pretrained(LLAMA, cache_dir=cache_dir)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

### 📂 Configuración de Caché de HuggingFace

**Variables de entorno configuradas en el contenedor:**

- `HF_HOME=/root/.cache/huggingface` - Directorio base de caché
- `TRANSFORMERS_CACHE=/root/.cache/huggingface/transformers` - Modelos Transformers
- `HUGGINGFACE_HUB_CACHE=/root/.cache/huggingface/hub` - HuggingFace Hub

**✅ Buenas prácticas:**

1. **Opción 1 (Recomendada)**: Omitir `cache_dir` - HuggingFace usa `HF_HOME` automáticamente
   ```python
   model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
   ```

2. **Opción 2**: Usar variable de entorno explícitamente
   ```python
   import os
   cache_dir = os.getenv('HF_HOME', '/root/.cache/huggingface')
   model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, cache_dir=cache_dir)
   ```

3. **❌ Evitar**: Rutas hardcodeadas de Windows
   ```python
   # MAL - No portable
   cache_dir = r"D:\dockerVolumes\hf_cache"
   ```

**Ventajas:**
- ✅ Código portable entre Docker, Colab, local
- ✅ Configuración centralizada en `docker-compose.yml`
- ✅ Todos los modelos persisten en `D:/dockerVolumes/hf_cache/`

In [ ]:
# The model
# Usar variable de entorno HF_HOME en lugar de ruta hardcodeada
import os
cache_dir = os.getenv('HF_HOME', '/root/.cache/huggingface')

model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config, cache_dir=cache_dir)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [10]:
memory = model.get_memory_footprint() / 1e6
print(f"Memory footprint: {memory:,.1f} MB")
#

Memory footprint: 5,591.5 MB


### Comparación: Memoria Original vs Cuantizada

El `memory footprint` muestra la memoria GPU/RAM que ocupa el modelo **después de cuantizar**:

- **Llama-3.1-8B sin cuantizar**: ~16,000 MB (16 GB)
- **Llama-3.1-8B con 4-bit**: ~4,500-5,000 MB (4.5-5 GB)

**Ahorro**: 70-75% de memoria

In [11]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRM

In [12]:
outputs = model.generate(inputs, max_new_tokens=80)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Eres un asistente util<|eot_id|><|start_header_id|>user<|end_header_id|>

Cuentame un chiste divertido para una sala llena de cientificos de datos<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Un científico de datos entra en un bar y dice al bartender: "Tengo una pregunta para ti. Si tienes 100 clientes y cada uno de ellos tiene una probabilidad del 1% de ordenar un whisky, ¿cuál es la probabilidad de que al menos un cliente ordene un whisky?"

El bartender responde: "Eso es fácil,


In [13]:
# Limpiar

del inputs, outputs, model
torch.cuda.empty_cache()

### Gestión de Almacenamiento en Docker

**¿Qué guardar para evitar re-descargas?**

Solo necesitas persistir el **caché de HuggingFace** con los modelos originales:

```
~/.cache/huggingface/hub/models--meta-llama--Llama-3.1-8B-Instruct/
```

**Solución recomendada**: Monta un volumen Docker para el caché:

```bash
docker run -v /ruta/host/cache:/root/.cache/huggingface \
           -v /workspace:/workspace \
           tu-imagen
```

**NO necesitas guardar**:
- ❌ El modelo cuantizado (se recrea en memoria cada vez)
- ❌ Los outputs de inferencia (se generan on-demand)

**SÍ necesitas guardar**:
- ✅ Caché de modelos originales (~16GB por modelo)
- ✅ Tu código/notebooks
- ✅ Variables de entorno (.env)

In [ ]:
# OPCIONAL: Guardar el modelo cuantizado manualmente para re-uso rápido
# Solo útil si vas a usar el mismo modelo cuantizado repetidamente

# Opción 1: Guardar con save_pretrained (guardará en formato cuantizado si está soportado)
# save_path = "/workspace/models_quantized/llama-3.1-8b-4bit"
# model.save_pretrained(save_path)
# tokenizer.save_pretrained(save_path)

# Opción 2: Verificar ubicación del caché original
from transformers import file_utils
import os

# En entornos Colab/Docker sin configuración personalizada
cache_dir = os.path.expanduser("~/.cache/huggingface/hub")
print(f"Caché de modelos HuggingFace: {cache_dir}")
print(f"Tamaño típico Llama-3.1-8B original: ~16 GB")
print(f"Tamaño típico cuantizado 4-bit en memoria: ~4-5 GB")

# Listar modelos descargados (si existen)
if os.path.exists(cache_dir):
    models = [d for d in os.listdir(cache_dir) if d.startswith("models--")]
    print(f"\nModelos descargados ({len(models)}): {models[:3]}")  # Primeros 3

## 🔴 Solución: Error "Torch not compiled with CUDA enabled"

Este error indica que PyTorch está instalado **sin soporte GPU**. Necesitas reinstalar PyTorch con CUDA.

### Diagnóstico Rápido

Ejecuta esta celda para verificar el estado actual:

In [1]:
# Diagnóstico: Verificar instalación de PyTorch y CUDA
import torch
import sys

print("=" * 60)
print("DIAGNÓSTICO PyTorch & CUDA")
print("=" * 60)
print(f"Python version: {sys.version.split()[0]}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"cuDNN version: {torch.backends.cudnn.version()}")
    print(f"Devices detectados: {torch.cuda.device_count()}")
    print(f"GPU actual: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ CUDA NO DISPONIBLE - PyTorch instalado en modo CPU")
    print("\n🔧 SOLUCIÓN: Reinstalar PyTorch con soporte CUDA")
    print("   Ver celda siguiente para instrucciones")
print("=" * 60)

DIAGNÓSTICO PyTorch & CUDA
Python version: 3.11.14
PyTorch version: 2.7.1+cu118
CUDA disponible: True
CUDA version: 11.8
cuDNN version: 90100
Devices detectados: 1
GPU actual: NVIDIA GeForce RTX 2080 Ti


### ✅ Verificación Final: PyTorch con CUDA en Todos los Entornos

Ejecuta la siguiente celda para confirmar que PyTorch con CUDA está instalado correctamente.

In [2]:
# Verificación completa de PyTorch con CUDA
import torch
import sys

print("=" * 70)
print("VERIFICACIÓN PYTORCH + CUDA - ENTORNO ACTUAL")
print("=" * 70)
print(f"Entorno Python: {sys.executable}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✅ CUDA version: {torch.version.cuda}")
    print(f"✅ cuDNN version: {torch.backends.cudnn.version()}")
    print(f"✅ GPU detectada: {torch.cuda.get_device_name(0)}")
    print(f"✅ Memoria GPU total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    
    # Test rápido
    tensor_cpu = torch.randn(3, 3)
    tensor_gpu = tensor_cpu.to('cuda')
    print(f"✅ Test GPU exitoso: tensor movido a {tensor_gpu.device}")
else:
    print("❌ CUDA NO DISPONIBLE")
    print("   Ejecuta las celdas de reinstalación anteriores")

print("=" * 70)

VERIFICACIÓN PYTORCH + CUDA - ENTORNO ACTUAL
Entorno Python: /opt/conda/envs/LLM/bin/python
PyTorch version: 2.7.1+cu118
CUDA disponible: True
✅ CUDA version: 11.8
✅ cuDNN version: 90100
✅ GPU detectada: NVIDIA GeForce RTX 2080 Ti
✅ Memoria GPU total: 11.81 GB
✅ Test GPU exitoso: tensor movido a cuda:0
